In [1]:
import pandas as pd
import numpy as np
import pyodbc
import warnings
import datetime
from datetime import timedelta
import logging


warnings.filterwarnings('ignore')

# for logging purposes
now = datetime.datetime.now()
print("Start Time : ", now)


Start Time :  2022-03-08 12:58:40.966309


In [5]:
# importing heat_analysis
#heat_analysis = pd.read_sql("select TOP(1000) * from heat_analysis order by MSG_TIME_STAMP desc", conn)

heat_analysis = pd.read_excel(r'C:\Users\khera siddharth\Desktop\Desktop_Folder\Case Work\JSW_dolvi\DeS\Working files\Model Prediction\March 8\input\DeS Input.xlsx', sheet_name='heat analysis')

# importing lf_heat_data
lf_heat_data = pd.read_excel(r'C:\Users\khera siddharth\Desktop\Desktop_Folder\Case Work\JSW_dolvi\DeS\Working files\Model Prediction\March 8\input\DeS Input.xlsx', sheet_name='lf data')

# importing the grade mapping fact table
grade_mapping = pd.read_excel(r'C:\Users\khera siddharth\Desktop\Desktop_Folder\Case Work\JSW_dolvi\DeS\Working files\Model Prediction\March 8\input\DeS Input.xlsx', sheet_name='grade mapping')

actual_al = pd.read_excel(r'C:\Users\khera siddharth\Desktop\Desktop_Folder\Case Work\JSW_dolvi\DeS\Working files\Model Prediction\March 8\input\DeS Input.xlsx', sheet_name='result')

nodes = pd.read_excel(r'C:\Users\khera siddharth\Desktop\Desktop_Folder\Case Work\JSW_dolvi\DeS\Working files\Model Prediction\March 8\input\DeS Input.xlsx', sheet_name='nodes')

######################### data processing ##############################

print(len(heat_analysis))
print(len(lf_heat_data))

# calculating required columns
lf_heat_data['final_O2_ppm'] = lf_heat_data['TAP_O2'] + lf_heat_data['O2AFTERCELOX']

# converting ALBAR to tons
lf_heat_data['ALBAR'] = lf_heat_data['ALBAR']/19

# renaming the required columns
lf_heat_data.rename(columns = {
    'GRADE_TYPE':'GRADE',
    'LM_START_WT':'LM_Start_Wt_lf',
    'FIRSTMEASTEMP':'1st_Probe_temp_lf',
    'ALBAR':'Al_Bar_lf',
    'LIME':'Lime_tap',
    'SIMN':'SiMn_tap',
    'LTA':'LTA_lf'
}, inplace = True)

# keeping only the required rows
lf_heat_data = lf_heat_data.groupby(['HEAT_NUMBER']).agg({'MSG_TIME_STAMP':'max','GRADE': 'first','1st_Probe_temp_lf':'max', 'Al_Bar_lf':'max','LM_Start_Wt_lf':'max','LTA_lf':'max','Lime_tap':'max','SiMn_tap':'max','final_O2_ppm':'max'})
lf_heat_data.reset_index(inplace = True)
lf_heat_data.sort_values(by = ['MSG_TIME_STAMP'], ascending = False, inplace = True)
lf_heat_data.drop_duplicates(subset = ['HEAT_NUMBER'], keep = 'first', inplace = True)

# preprocessing heat_analysis
# if there are multiple records for a heat, keeping only the latest one
heat_analysis.drop_duplicates(subset = ['HEAT_NUMBER'], keep = 'first', inplace = True)

# keeping only required columns
heat_analysis = heat_analysis[['HEAT_NUMBER','AGGREGATE','AL_TOTAL','S','C','SI','Month']]

# renaming columns as per requirement
heat_analysis.rename(columns = {'AL_TOTAL':'AL_chem_first', 'S':'S_chem_first','SI':'SI_chem_first','C':'C_chem_first'}, inplace = True)


# merging sample chemistry data with lf data
ads = heat_analysis.merge(lf_heat_data, on = ['HEAT_NUMBER'], how = 'inner')
print(len(ads))


ads['LOI_perc'] = 6

# mapping the grades to their grade type (CG/CR or VAG)
# the grades that don't get mapped are billet heats, and so will not be run in this model
ads = ads.merge(grade_mapping, on = ['GRADE'], how = 'inner')

print(len(ads))

15830
45142
10781
10741


In [6]:
ads_test=ads.copy()
#ads_test=ads_test[(ads_test['Month']==12) | (ads_test['Month']==1) | (ads_test['Month']==2)]
 
ads_test=ads_test.merge(nodes, on ='HEAT_NUMBER', how ='left')
print(len(ads_test))
ads_test=ads_test[ads_test['GRADE_TYPE']=='VAG']
print(len(ads_test))
# ads_test=ads_test[ads_test['Carbon_Node']<=5]


10806
5003


In [15]:
model_ads=ads_test.copy()
print(len(model_ads))

model_ads=model_ads[(model_ads['final_O2_ppm']>700) & (model_ads['final_O2_ppm']<=1450)]
print(len(model_ads))
model_ads=model_ads[(model_ads['LM_Start_Wt_lf']>=170) & (model_ads['LM_Start_Wt_lf']<=205)]
print(len(model_ads))
#model_ads=model_ads[(model_ads['Al_Bar_lf']<=25) & (model_ads['Al_Bar_lf']>=15)]
#print(len(model_ads))
print(len(model_ads[model_ads['Lime_tap']>=700]))
#print(len(model_ads[model_ads['AL_chem_first']<=750]))
#print(len(model_ads[model_ads['S_chem_first']<=750]))

5003
4215
3787
2579


In [ ]:
#actual_al = pd.read_excel(r'C:\Users\khera siddharth\Desktop\Desktop_Folder\Case Work\JSW_dolvi\DeS\Working files\Model Prediction\Feb 22\input\DeS Input.xlsx', sheet_name='result')


In [16]:

def outlier_treatment(df,col_list,lv_imp,uv_imp):
    """
    
    Intended to treat outlier values based on defined thresholds lv and uv
    returns : dataframe with values treated for outliers
    """
    for col,lv,uv in zip(col_list,lv_imp,uv_imp):
        df[col+'_treated'] = [uv if x>uv else x for x in df[col]]
        df[col+'_treated'] = [lv if x<lv else x for x in df[col+'_treated']]
        
        #print(df[col+'_treated'])
    return df

def impute_missing(df,cols=[],simple_impute=True,impute_value=0):
    """
    
    Intended to impute missing values for the specified columns by the specified value
    returns : dataframe with imputed values
    """
    if simple_impute == 1:
            if len(cols)>0:
                df[cols] = df[cols].fillna(value=impute_value)
            else:
                df.fillna(impute_value,inplace=True)
    else:
        df.fillna(df[cols].median(),inplace=True)
        print("Column and Median to be replace : ",cols, df[cols].median())
    
    return df

def univariate(df):
    """
     
    Intended to provide a percentile distribution for numeric columns and 
    frenquency counts for categorical variables
    returns : data frame with univariates for each column
    """
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64','object']
    quantiles = [0,0.001,0.01,0.05,0.1,0.2,0.25,0.3,0.4,0.5,0.6,0.7,0.75,0.8,0.9,0.95,0.99,0.999,1]
    
    df_num = pd.DataFrame(df.select_dtypes(include=numerics).quantile(quantiles).transpose())
    num_cols = df_num.index.tolist()
    char_cols = [cols for cols in df.columns if cols not in num_cols]
    
    """
    df_char = df[char_cols]
    df_cat_agg = pd.DataFrame()
    
    # Frequency counts for char columns 
    if len(char_cols) > 0:
        count = 0
        for col in char_cols:
            df_ = df_char.groupby(col).agg(['count'])
            df_['column_name'] = col
            df_cat_agg = df_cat_agg.append(df_)      
    
    #return df_num,df_cat_agg
    """
    
    return df_num

features = ['final_O2_ppm', 'LM_Start_Wt_lf',
           '1st_Probe_temp_lf','Al_Bar_lf', 'Lime_tap',
           'AL_chem_first', 'S_chem_first']

# model_ads=ads_test.copy()
model_ads=model_ads.merge(actual_al[['HEAT_NUMBER','Actual Al']].rename(columns={'Actual Al':'AL_mat'}),on='HEAT_NUMBER', how='left')
model_ads = model_ads[model_ads['AL_mat'].notna()]
#model_ads = model_ads[model_ads['AL_mat']!=0]

print(len(model_ads))
model_ads = model_ads[model_ads['AL_mat']<=600]
print(len(model_ads))


cols_null_value_treatment=['final_O2_ppm', 'LM_Start_Wt_lf',
           '1st_Probe_temp_lf','Al_Bar_lf', 'Lime_tap',
           'AL_chem_first', 'S_chem_first','AL_mat']

model_ads = impute_missing(model_ads, cols=cols_null_value_treatment,simple_impute=False)
print(len(model_ads))

# Outlier treatment
cols_of_interest=['AL_chem_first','S_chem_first','Lime_tap','Al_Bar_lf']
# feature_cols = [x for x in cols_of_interest if x not in ['Heat','AL_mat']]
univar_num = univariate(model_ads[['AL_chem_first','S_chem_first','Lime_tap','Al_Bar_lf']])


lv = univar_num[0.001].to_list()
uv = univar_num[0.99].to_list()
# Treat Outliers

model_ads = outlier_treatment(model_ads,cols_of_interest,lv,uv)

print(len(model_ads))

# features = ['final_O2_ppm', 'LM_Start_Wt_lf',
#            '1st_Probe_temp_lf','Al_Bar_lf', 'Lime_tap',
#            'AL_chem_first', 'S_chem_first']
# features = ['final_O2_ppm_treated', 'LM_Start_Wt_lf_treated',
#            '1st_Probe_temp_lf_treated','Al_Bar_lf_treated', 'Lime_tap_treated',
#            'AL_chem_first_treated', 'S_chem_first_treated']

#model_ads[['final_O2_ppm', 'LM_Start_Wt_lf','1st_Probe_temp_lf','Al_Bar_lf', 'Lime_tap','AL_chem_first', 'S_chem_first']]=model_ads[['final_O2_ppm_treated', 'LM_Start_Wt_lf_treated','1st_Probe_temp_lf_treated','Al_Bar_lf_treated', 'Lime_tap_treated','AL_chem_first_treated', 'S_chem_first_treated']]
model_ads[['Lime_tap','AL_chem_first', 'S_chem_first','Al_Bar_lf']]=model_ads[['Lime_tap_treated','AL_chem_first_treated', 'S_chem_first_treated','Al_Bar_lf_treated']]



# Setting target variable
target = ['AL_mat']


2890
2876
Column and Median to be replace :  ['final_O2_ppm', 'LM_Start_Wt_lf', '1st_Probe_temp_lf', 'Al_Bar_lf', 'Lime_tap', 'AL_chem_first', 'S_chem_first', 'AL_mat'] final_O2_ppm         1124.000000
LM_Start_Wt_lf        189.000000
1st_Probe_temp_lf    1567.000000
Al_Bar_lf              17.368421
Lime_tap              996.500000
AL_chem_first           0.031000
S_chem_first            0.024000
AL_mat                125.400000
dtype: float64
2876
2876


In [19]:
model_ads['Al_Bar_lf']=model_ads['Al_Bar_lf'].clip(15)
model_ads['Al_Bar_lf'].describe()


count    2876.000000
mean       18.443836
std         4.016258
min        15.000000
25%        15.000000
50%        17.368421
75%        20.842105
max        39.526316
Name: Al_Bar_lf, dtype: float64

In [20]:
model_ads_train_test_df=model_ads.copy()

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
# Split the training set to train and evaluate samples
X_train, X_test, y_train, y_test = train_test_split(model_ads_train_test_df[features],
                                                    model_ads_train_test_df[target],
                                                    test_size=0.25,
                                                    random_state=97)

#Merging the datasets
train_df = pd.concat([X_train,y_train], axis = 1).reset_index(drop=True)
test_df = pd.concat([X_test,y_test], axis = 1).reset_index(drop=True)

def train_decision_tree(df_features,df_target,n_=200,leaf_=10,split_=25,md_=4, random_state = 97):
    
    # Initialise dataframe 
    weights=pd.DataFrame(columns=['Variable','Importance_gain','iteration','rand_index'])
    
    # Instantiate the model with required parameters
    model_=DecisionTreeRegressor(max_depth=md_, random_state=random_state,criterion='mse',min_samples_leaf=leaf_, 
                                    min_samples_split=split_)    
        
    print(model_)
    model_.fit(df_features,df_target)
    weights = pd.DataFrame(model_.feature_importances_,
                                   index = df_features.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)   
    return model_,weights

def mape(y_pred,y_true,label,key): 
    mape_calc = pd.DataFrame(data=np.column_stack((y_pred,y_true,key)),columns=['predicted','actual','key'])
    mape_calc['mape'] = abs(mape_calc['predicted']-mape_calc['actual'])
    mape_calc = mape_calc[mape_calc['actual']>0]
    #mape_calc.to_csv(f"{output_path}\\"+label+f'_mape_validate_{now}.csv')
    
    return mape_calc['mape'].mean()


# Running Decision Tree
model_rf,weights_rf = train_decision_tree(X_train,y_train,leaf_=10,split_=25,md_=4,random_state = 97)

# Predicting on the train and test data
pred_train_rf = model_rf.predict(X_train)
pred_test_rf = model_rf.predict(X_test)

print("MAPE for train is {}ppm".format(round(mape(pred_train_rf,y_train.values,'train',y_train.index),2)))
print("MAPE for test is {}ppm".format(round(mape(pred_test_rf,y_test.values,'test',y_test.index),2))) 

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=4,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=10, min_samples_split=25,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=97, splitter='best')
MAPE for train is 55.58ppm
MAPE for test is 55.25ppm


In [21]:
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
            p1 += [f"({name} <= {np.round(threshold, 3)})"]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [f"({name} > {np.round(threshold, 3)})"]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = "if "
        
        for p in path[:-1]:
            if rule != "if ":
                rule += " and "
            rule += str(p)
        rule += " then "
        if class_names is None:
            rule += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            rule += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
        rule += f" | based on {path[-1][1]:,} samples"
        rules += [rule]
        
    return rules

In [22]:
# Print rules
from sklearn.tree import _tree
rules = get_rules(model_rf, features, None)

In [23]:
df = pd.DataFrame(columns = ["1", "2","3","4","5","6","7"])
for r in rules:
    str_to_append = str(r).split('and')
    str_to_append
    df_length = len(df)
    if len(str_to_append) < 4:
        str_to_append.insert(2, "")
        str_to_append.insert(3, str_to_append[3].split("then")[0])
        str_to_append.insert(4, str_to_append[4].split("then")[1].split("|")[0])
        str_to_append.insert(5, str_to_append[5].split("then")[1].split("|")[1])
        df.loc[df_length] = str_to_append
    else:
        str_to_append.insert(3, str_to_append[3].split("then")[0])
        str_to_append.insert(4, str_to_append[4].split("then")[1].split("|")[0])
        str_to_append.insert(5, str_to_append[5].split("then")[1].split("|")[1])
        df.loc[df_length] = str_to_append    

In [24]:
# Keeping columns of interest
df_flt = df[['1', '2', '3', '4', '5', '6']]

In [25]:
def replace_strings(row):
    """
    
    This function will replace the strings in the columns
    """
    row['1'] = row['1'].replace('if (','')
    row['1'] = row['1'].replace(') ','')
    row['2'] = row['2'].replace(' (','')
    row['2'] = row['2'].replace(') ','')
    row['3'] = row['3'].replace(' (','')
    row['3'] = row['3'].replace(') ','')
    row['4'] = row['4'].replace(' (','')
    row['4'] = row['4'].replace(') ','')
    row['5'] = row['5'].replace(' response: ','')
    row['5'] = row['5'].replace(' ','')
    row['6'] = row['6'].replace(' based on ','')
    row['6'] = row['6'].replace(' samples','')
    
    return row

In [26]:
# Calling the function to run it
df_flt = df_flt.apply(replace_strings, axis =1)

In [27]:
# Renaming the columns
df_flt.columns = ['Split1','Split2','Split3','Split4','Suggested_value','No_of_Samples']

In [38]:
dt_rules=df_flt.copy()
dt_rules['Split3'][dt_rules['Split3']==""]=dt_rules['Split2']

dt_rules


,Split1,Split2,Split3,Split4,Suggested_value,No_of_Samples
0,Al_Bar_lf > 15.026,AL_chem_first > 0.046,Lime_tap <= 2927.5,S_chem_first <= 0.027,145.698,483
1,Al_Bar_lf > 15.026,AL_chem_first <= 0.046,S_chem_first > 0.02,final_O2_ppm > 980.5,253.355,313
2,Al_Bar_lf <= 15.026,AL_chem_first <= 0.032,S_chem_first > 0.027,S_chem_first <= 0.035,68.62,284
3,Al_Bar_lf > 15.026,AL_chem_first <= 0.046,S_chem_first <= 0.02,Lime_tap <= 3088.5,169.827,283
4,Al_Bar_lf <= 15.026,AL_chem_first <= 0.032,S_chem_first <= 0.027,final_O2_ppm <= 1144.5,28.306,203
5,Al_Bar_lf <= 15.026,AL_chem_first <= 0.032,S_chem_first > 0.027,S_chem_first > 0.035,92.077,176
6,Al_Bar_lf > 15.026,AL_chem_first > 0.046,Lime_tap <= 2927.5,S_chem_first > 0.027,188.458,141
7,Al_Bar_lf <= 15.026,AL_chem_first > 0.032,Lime_tap > 995.5,S_chem_first <= 0.029,171.716,82
8,Al_Bar_lf > 15.026,AL_chem_first <= 0.046,S_chem_first > 0.02,final_O2_ppm <= 980.5,175.593,55
9,Al_Bar_lf <= 15.026,AL_chem_first <= 0.032,S_chem_first <= 0.027,final_O2_ppm > 1144.5,89.132,50


In [39]:
# reading the ads file

# filling null rules
dt_rules['Split3'].fillna(dt_rules["Split2"], inplace = True)

# keeping only the split1, split2, split3 & split4 cols
dt_rules = dt_rules[['Split1','Split2','Split3','Split4']]

# extracting rule information from string
def get_rules_from_string(rule):
    col = rule.split(' ')[0]
    ope = rule.split(' ')[1]
    val = rule.split(' ')[2]
    
    return col, ope, val

# print rules from rule df
def print_rules(df_name = 'model_ads_train_test_df', rules = dt_rules):
    
    main_rules = ''
    for i in range(len(dt_rules)):
        
        df_sub = f'df{i+1}'
        condition_string = ''
        for j in range(0,4):
            col,ope,val = get_rules_from_string(dt_rules.iloc[i,j])
            split_string = '(' + f"{df_name}['{col}'] {ope} {val}" + ')'
            
            if j != 3:
                split_string = split_string + ' & '
            condition_string = condition_string + split_string
            
        rule_string = f"{df_sub} = {df_name}[{condition_string}]"
        main_rules = main_rules + '\n' + rule_string
    
    return main_rules

# print rules from rule df
def get_splits(df_name = 'row', rules = dt_rules):
    
    main_rules = []
    for i in range(len(dt_rules)):
        
        df_sub = f'df{i}'
        condition_string = ''
        for j in range(0,4):
            col,ope,val = get_rules_from_string(dt_rules.iloc[i,j])
            split_string = '(' + f"{df_name}['{col}'] {ope} {val}" + ')'
            
            if j != 3:
                split_string = split_string + ' and '
            condition_string = condition_string + split_string
            
        rule_string = f"elif {condition_string}:"
        main_rules.append(rule_string)
    
    return main_rules

# getting rules
rule_string = print_rules()
# getting rule splits for report
splits = get_splits()

    
# printing here so that user can directly copy
print(rule_string)


df1 = model_ads_train_test_df[(model_ads_train_test_df['Al_Bar_lf'] > 15.026) & (model_ads_train_test_df['AL_chem_first'] > 0.046) & (model_ads_train_test_df['Lime_tap'] <= 2927.5) & (model_ads_train_test_df['S_chem_first'] <= 0.027)]
df2 = model_ads_train_test_df[(model_ads_train_test_df['Al_Bar_lf'] > 15.026) & (model_ads_train_test_df['AL_chem_first'] <= 0.046) & (model_ads_train_test_df['S_chem_first'] > 0.02) & (model_ads_train_test_df['final_O2_ppm'] > 980.5)]
df3 = model_ads_train_test_df[(model_ads_train_test_df['Al_Bar_lf'] <= 15.026) & (model_ads_train_test_df['AL_chem_first'] <= 0.032) & (model_ads_train_test_df['S_chem_first'] > 0.027) & (model_ads_train_test_df['S_chem_first'] <= 0.035)]
df4 = model_ads_train_test_df[(model_ads_train_test_df['Al_Bar_lf'] > 15.026) & (model_ads_train_test_df['AL_chem_first'] <= 0.046) & (model_ads_train_test_df['S_chem_first'] <= 0.02) & (model_ads_train_test_df['Lime_tap'] <= 3088.5)]
df5 = model_ads_train_test_df[(model_ads_train_test_df

In [41]:
df1 = model_ads_train_test_df[(model_ads_train_test_df['Al_Bar_lf'] > 15.026) & (model_ads_train_test_df['AL_chem_first'] > 0.046) & (model_ads_train_test_df['Lime_tap'] <= 2927.5) & (model_ads_train_test_df['S_chem_first'] <= 0.027)]
df2 = model_ads_train_test_df[(model_ads_train_test_df['Al_Bar_lf'] > 15.026) & (model_ads_train_test_df['AL_chem_first'] <= 0.046) & (model_ads_train_test_df['S_chem_first'] > 0.02) & (model_ads_train_test_df['final_O2_ppm'] > 980.5)]
df3 = model_ads_train_test_df[(model_ads_train_test_df['Al_Bar_lf'] <= 15.026) & (model_ads_train_test_df['AL_chem_first'] <= 0.032) & (model_ads_train_test_df['S_chem_first'] > 0.027) & (model_ads_train_test_df['S_chem_first'] <= 0.035)]
df4 = model_ads_train_test_df[(model_ads_train_test_df['Al_Bar_lf'] > 15.026) & (model_ads_train_test_df['AL_chem_first'] <= 0.046) & (model_ads_train_test_df['S_chem_first'] <= 0.02) & (model_ads_train_test_df['Lime_tap'] <= 3088.5)]
df5 = model_ads_train_test_df[(model_ads_train_test_df['Al_Bar_lf'] <= 15.026) & (model_ads_train_test_df['AL_chem_first'] <= 0.032) & (model_ads_train_test_df['S_chem_first'] <= 0.027) & (model_ads_train_test_df['final_O2_ppm'] <= 1144.5)]
df6 = model_ads_train_test_df[(model_ads_train_test_df['Al_Bar_lf'] <= 15.026) & (model_ads_train_test_df['AL_chem_first'] <= 0.032) & (model_ads_train_test_df['S_chem_first'] > 0.027) & (model_ads_train_test_df['S_chem_first'] > 0.035)]
df7 = model_ads_train_test_df[(model_ads_train_test_df['Al_Bar_lf'] > 15.026) & (model_ads_train_test_df['AL_chem_first'] > 0.046) & (model_ads_train_test_df['Lime_tap'] <= 2927.5) & (model_ads_train_test_df['S_chem_first'] > 0.027)]
df8 = model_ads_train_test_df[(model_ads_train_test_df['Al_Bar_lf'] <= 15.026) & (model_ads_train_test_df['AL_chem_first'] > 0.032) & (model_ads_train_test_df['Lime_tap'] > 995.5) & (model_ads_train_test_df['S_chem_first'] <= 0.029)]
df9 = model_ads_train_test_df[(model_ads_train_test_df['Al_Bar_lf'] > 15.026) & (model_ads_train_test_df['AL_chem_first'] <= 0.046) & (model_ads_train_test_df['S_chem_first'] > 0.02) & (model_ads_train_test_df['final_O2_ppm'] <= 980.5)]
df10 = model_ads_train_test_df[(model_ads_train_test_df['Al_Bar_lf'] <= 15.026) & (model_ads_train_test_df['AL_chem_first'] <= 0.032) & (model_ads_train_test_df['S_chem_first'] <= 0.027) & (model_ads_train_test_df['final_O2_ppm'] > 1144.5)]
df11 = model_ads_train_test_df[(model_ads_train_test_df['Al_Bar_lf'] <= 15.026) & (model_ads_train_test_df['AL_chem_first'] > 0.032) & (model_ads_train_test_df['Lime_tap'] <= 995.5) & (model_ads_train_test_df['LM_Start_Wt_lf'] > 185.5)]
df12 = model_ads_train_test_df[(model_ads_train_test_df['Al_Bar_lf'] <= 15.026) & (model_ads_train_test_df['AL_chem_first'] > 0.032) & (model_ads_train_test_df['Lime_tap'] > 995.5) & (model_ads_train_test_df['S_chem_first'] > 0.029)]
df13 = model_ads_train_test_df[(model_ads_train_test_df['Al_Bar_lf'] <= 15.026) & (model_ads_train_test_df['AL_chem_first'] > 0.032) & (model_ads_train_test_df['Lime_tap'] <= 995.5) & (model_ads_train_test_df['LM_Start_Wt_lf'] <= 185.5)]
df14 = model_ads_train_test_df[(model_ads_train_test_df['Al_Bar_lf'] > 15.026) & (model_ads_train_test_df['AL_chem_first'] > 0.046) & (model_ads_train_test_df['AL_chem_first'] > 0.046) & (model_ads_train_test_df['Lime_tap'] > 2927.5)]
df15 = model_ads_train_test_df[(model_ads_train_test_df['Al_Bar_lf'] > 15.026) & (model_ads_train_test_df['AL_chem_first'] <= 0.046) & (model_ads_train_test_df['S_chem_first'] <= 0.02) & (model_ads_train_test_df['Lime_tap'] > 3088.5)]

In [72]:
# Creating a list of dataset
df_list = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15]#,df16]

In [79]:
y_delta=abs(y_true-y_pred)

In [82]:
y_delta

array([5.95805826e+01, 2.76488185e+01, 3.64892148e+01, 1.31629831e+02,
       1.33407144e+02, 8.42683545e+01, 1.54754267e+02, 3.53117713e+01,
       7.85169993e+01, 4.27953367e+01, 1.08433892e+02, 4.76867647e+01,
       7.40013449e+01, 1.43353368e-01])

In [92]:
#y_new=
100*sum([1 for i in np.where(y_delta <= 40)[0]])/len(y_true)

28.571428571428573

In [90]:
len(y_true)

14

In [44]:
import statsmodels.api as sm

In [94]:
# 'equations' is used to print the rules for the model which can be plugged into the optimization code
equations = []
mapes = []
r2s = []
for i in df_list:
    # Put the target (housing value -- MEDV) in another DataFrame
    X = i[features]
    y = i[["AL_mat"]]

    # Note the difference in argument order
    model = sm.OLS(y, X).fit()
    
    predictions = model.predict(X) # make the predictions by the model
    
    # Mape calculation
    y_true, y_pred = np.array(y), np.array(predictions)

    y_pred=[y_pred[i] for i in np.where(y_true != 0)[0]]
    y_true=y_true[y_true!=0]
    
    y_delta=abs(y_true-y_pred)
    
    print('len :',len(y_true)    )
    print("Accuracy @ 50 : ", sum([1 for i in np.where(y_delta <= 50)[0]])*100/len(y_true))    
    print("Accuracy @ 40 : ", sum([1 for i in np.where(y_delta <= 40)[0]])*100/len(y_true))

    print("MAPE : ", np.mean(np.abs((y_true - y_pred) / y_true)) * 100)
    mapes.append("MAPE : " + str(np.mean(np.abs((y_true - y_pred) / y_true)) * 100))

    # Print out the statistics
    #model.summary()
    
    # Calculating R Square
    r_squared_table = pd.read_html(model.summary().tables[0].as_html(),header=0,index_col=0)[0]
    r_squared_table.columns = ['1','2','3']
    try:
        adj_r_square = float(r_squared_table[r_squared_table['2']=='Adj. R-squared (uncentered):']['3']) * 100
    except:
        adj_r_square = float(r_squared_table[r_squared_table['2']=='Adj. R-squared:']['3']) * 100

    # Saving R2s
    r2s.append("Adjusted RSquared : " + str(adj_r_square))
    
    # Getting the coefficient table
    coefficient_table = pd.read_html(model.summary().tables[1].as_html(),header=0,index_col=0)[0]

    # Resetting the index
    coefficient_table = coefficient_table.reset_index()

    # Keeping columns of interest
    coefficient_table = coefficient_table[['index','coef']]

    equation = ''
    for i in range(coefficient_table.shape[0]):
        if i == 0:
            equation = str(coefficient_table['coef'][i]) + ' * ' + "row[\'" +str(coefficient_table['index'][i]) +"\']"
        else:
            equation = equation + " + " + str(coefficient_table['coef'][i]) + ' * ' + 'row[\'' + str(coefficient_table['index'][i]) + '\']'
    
    # saving equations
    equations.append("row['AL_mat'] = " + equation)
    
    print("Adjusted R2 : ",adj_r_square)       
    print(equation)

len : 625
Accuracy @ 50 :  58.72
Accuracy @ 40 :  48.8
MAPE :  48.148742165381094
Adjusted R2 :  80.0
-0.0324 * row['final_O2_ppm'] + 0.7724 * row['LM_Start_Wt_lf'] + 0.0168 * row['1st_Probe_temp_lf'] + 1.231 * row['Al_Bar_lf'] + 0.0402 * row['Lime_tap'] + -1515.8216 * row['AL_chem_first'] + 1878.593 * row['S_chem_first']
len : 410
Accuracy @ 50 :  53.65853658536585
Accuracy @ 40 :  44.390243902439025
MAPE :  46.11808527667793
Adjusted R2 :  86.0
-0.0825 * row['final_O2_ppm'] + 0.8239 * row['LM_Start_Wt_lf'] + 0.0801 * row['1st_Probe_temp_lf'] + 2.2775 * row['Al_Bar_lf'] + -0.0011 * row['Lime_tap'] + -1947.7474 * row['AL_chem_first'] + 2673.3038 * row['S_chem_first']
len : 330
Accuracy @ 50 :  90.3030303030303
Accuracy @ 40 :  85.45454545454545
MAPE :  30.096139755379625
Adjusted R2 :  12.2
0.0438 * row['final_O2_ppm'] + 0.6855 * row['LM_Start_Wt_lf'] + 0.3155 * row['1st_Probe_temp_lf'] + -47.4434 * row['Al_Bar_lf'] + 0.0242 * row['Lime_tap'] + 1995.5641 * row['AL_chem_first'] + 2986.0

In [96]:
with open(f'model_report.txt','w') as file:
    
    # writing time period
    
    
        
    # writing rules for optimization python code
    file.write("\n ############## Rules for Optimization Python Code ################ \n")
    for i in range(len(splits)):
        file.write(splits[i])
        file.write('\n\t')
        file.write(equations[i])
        file.write('\n\t')
        file.write(f'row[\'node\'] = {i+1}')
        file.write('\n')
        
    

In [20]:
model_ads.to_excel(r'C:\Users\khera siddharth\Desktop\Desktop_Folder\Case Work\JSW_dolvi\DeS\Working files\Model Prediction\Feb 23\output\testing\model_testing.xlsx')

In [23]:
ads.to_excel(r'C:\Users\khera siddharth\Desktop\Desktop_Folder\Case Work\JSW_dolvi\DeS\Working files\Model Prediction\Feb 23\output\testing\all_data_aug_feb.xlsx')

In [98]:
def vag_node_check(row):
    """

    This function will calculate the aluminium material addition for CG/CR grades
    # the "node" columns are only for debugging purposes
    """
    # initializing to 0, will get replaced if we use oxygen tree
    row['Oxygen Node'] = 0

    if (row['Al_Bar_lf'] > 15.026) and (row['AL_chem_first'] > 0.046) and (row['Lime_tap'] <= 2927.5) and (row['S_chem_first'] <= 0.027):
        row['node'] = 1
    elif (row['Al_Bar_lf'] > 15.026) and (row['AL_chem_first'] <= 0.046) and (row['S_chem_first'] > 0.02) and (row['final_O2_ppm'] > 980.5):
        row['node'] = 2
    elif (row['Al_Bar_lf'] <= 15.026) and (row['AL_chem_first'] <= 0.032) and (row['S_chem_first'] > 0.027) and (row['S_chem_first'] <= 0.035):
        row['node'] = 3
    elif (row['Al_Bar_lf'] > 15.026) and (row['AL_chem_first'] <= 0.046) and (row['S_chem_first'] <= 0.02) and (row['Lime_tap'] <= 3088.5):
        row['node'] = 4
    elif (row['Al_Bar_lf'] <= 15.026) and (row['AL_chem_first'] <= 0.032) and (row['S_chem_first'] <= 0.027) and (row['final_O2_ppm'] <= 1144.5):
        row['node'] = 5
    elif (row['Al_Bar_lf'] <= 15.026) and (row['AL_chem_first'] <= 0.032) and (row['S_chem_first'] > 0.027) and (row['S_chem_first'] > 0.035):
        row['node'] = 6
    elif (row['Al_Bar_lf'] > 15.026) and (row['AL_chem_first'] > 0.046) and (row['Lime_tap'] <= 2927.5) and (row['S_chem_first'] > 0.027):
        row['node'] = 7
    elif (row['Al_Bar_lf'] <= 15.026) and (row['AL_chem_first'] > 0.032) and (row['Lime_tap'] > 995.5) and (row['S_chem_first'] <= 0.029):
        row['node'] = 8
    elif (row['Al_Bar_lf'] > 15.026) and (row['AL_chem_first'] <= 0.046) and (row['S_chem_first'] > 0.02) and (row['final_O2_ppm'] <= 980.5):
        row['node'] = 9
    elif (row['Al_Bar_lf'] <= 15.026) and (row['AL_chem_first'] <= 0.032) and (row['S_chem_first'] <= 0.027) and (row['final_O2_ppm'] > 1144.5):
        row['node'] = 10
    elif (row['Al_Bar_lf'] <= 15.026) and (row['AL_chem_first'] > 0.032) and (row['Lime_tap'] <= 995.5) and (row['LM_Start_Wt_lf'] > 185.5):
        row['node'] = 11
    elif (row['Al_Bar_lf'] <= 15.026) and (row['AL_chem_first'] > 0.032) and (row['Lime_tap'] > 995.5) and (row['S_chem_first'] > 0.029):
        row['node'] = 12
    elif (row['Al_Bar_lf'] <= 15.026) and (row['AL_chem_first'] > 0.032) and (row['Lime_tap'] <= 995.5) and (row['LM_Start_Wt_lf'] <= 185.5):
        row['node'] = 13
    elif (row['Al_Bar_lf'] > 15.026) and (row['AL_chem_first'] > 0.046) and (row['AL_chem_first'] > 0.046) and (row['Lime_tap'] > 2927.5):
        row['node'] = 14
    elif (row['Al_Bar_lf'] > 15.026) and (row['AL_chem_first'] <= 0.046) and (row['S_chem_first'] <= 0.02) and (row['Lime_tap'] > 3088.5):
        row['node'] = 15
    else:
        print("weak node")
        row['node'] = 0
        
    return row['node']

In [99]:
def vag_aluminium_prediction_new(row):
    """

    This function will calculate the aluminium material addition for CG/CR grades
    # the "node" columns are only for debugging purposes
    """

    al_lsl = 0.030
    fade_rate = 0.004

    # initializing to 0, will get replaced if we use oxygen tree
    row['Oxygen Node'] = 0

    if (row['Al_Bar_lf'] > 15.026) and (row['AL_chem_first'] > 0.046) and (row['Lime_tap'] <= 2927.5) and (row['S_chem_first'] <= 0.027):
        row['AL_mat'] = -0.0324 * row['final_O2_ppm'] + 0.7724 * row['LM_Start_Wt_lf'] + 0.0168 * row['1st_Probe_temp_lf'] + 1.231 * row['Al_Bar_lf'] + 0.0402 * row['Lime_tap'] + -1515.8216 * row['AL_chem_first'] + 1878.593 * row['S_chem_first']
        row['node'] = 1
    elif (row['Al_Bar_lf'] > 15.026) and (row['AL_chem_first'] <= 0.046) and (row['S_chem_first'] > 0.02) and (row['final_O2_ppm'] > 980.5):
        row['AL_mat'] = -0.0825 * row['final_O2_ppm'] + 0.8239 * row['LM_Start_Wt_lf'] + 0.0801 * row['1st_Probe_temp_lf'] + 2.2775 * row['Al_Bar_lf'] + -0.0011 * row['Lime_tap'] + -1947.7474 * row['AL_chem_first'] + 2673.3038 * row['S_chem_first']
        row['node'] = 2
    elif (row['Al_Bar_lf'] <= 15.026) and (row['AL_chem_first'] <= 0.032) and (row['S_chem_first'] > 0.027) and (row['S_chem_first'] <= 0.035):
        row['AL_mat'] = 0.0438 * row['final_O2_ppm'] + 0.6855 * row['LM_Start_Wt_lf'] + 0.3155 * row['1st_Probe_temp_lf'] + -47.4434 * row['Al_Bar_lf'] + 0.0242 * row['Lime_tap'] + 1995.5641 * row['AL_chem_first'] + 2986.0422 * row['S_chem_first']
        row['node'] = 3
    elif (row['Al_Bar_lf'] > 15.026) and (row['AL_chem_first'] <= 0.046) and (row['S_chem_first'] <= 0.02) and (row['Lime_tap'] <= 3088.5):
        row['AL_mat'] = 0.0555 * row['final_O2_ppm'] + 0.3062 * row['LM_Start_Wt_lf'] + -0.0246 * row['1st_Probe_temp_lf'] + -0.3171 * row['Al_Bar_lf'] + 0.0154 * row['Lime_tap'] + 746.5366 * row['AL_chem_first'] + 3383.4895 * row['S_chem_first']
        row['node'] = 4
    elif (row['Al_Bar_lf'] <= 15.026) and (row['AL_chem_first'] <= 0.032) and (row['S_chem_first'] <= 0.027) and (row['final_O2_ppm'] <= 1144.5):
        row['AL_mat'] = 0.0382 * row['final_O2_ppm'] + 0.3324 * row['LM_Start_Wt_lf'] + 0.0252 * row['1st_Probe_temp_lf'] + -10.1441 * row['Al_Bar_lf'] + 0.0239 * row['Lime_tap'] + 920.4385 * row['AL_chem_first'] + 896.9098 * row['S_chem_first']
        row['node'] = 5
    elif (row['Al_Bar_lf'] <= 15.026) and (row['AL_chem_first'] <= 0.032) and (row['S_chem_first'] > 0.027) and (row['S_chem_first'] > 0.035):
        row['AL_mat'] = 0.0176 * row['final_O2_ppm'] + 0.1702 * row['LM_Start_Wt_lf'] + -0.2343 * row['1st_Probe_temp_lf'] + 22.0835 * row['Al_Bar_lf'] + 0.004 * row['Lime_tap'] + 839.7014 * row['AL_chem_first'] + 1598.1017 * row['S_chem_first']
        row['node'] = 6
    elif (row['Al_Bar_lf'] > 15.026) and (row['AL_chem_first'] > 0.046) and (row['Lime_tap'] <= 2927.5) and (row['S_chem_first'] > 0.027):
        row['AL_mat'] = -0.0547 * row['final_O2_ppm'] + 1.4363 * row['LM_Start_Wt_lf'] + -0.089 * row['1st_Probe_temp_lf'] + 2.528 * row['Al_Bar_lf'] + 0.0229 * row['Lime_tap'] + -1716.6099 * row['AL_chem_first'] + 4898.4014 * row['S_chem_first']
        row['node'] = 7
    elif (row['Al_Bar_lf'] <= 15.026) and (row['AL_chem_first'] > 0.032) and (row['Lime_tap'] > 995.5) and (row['S_chem_first'] <= 0.029):
        row['AL_mat'] = 0.0525 * row['final_O2_ppm'] + -0.5727 * row['LM_Start_Wt_lf'] + -0.5461 * row['1st_Probe_temp_lf'] + 69.4802 * row['Al_Bar_lf'] + 0.0088 * row['Lime_tap'] + 455.8244 * row['AL_chem_first'] + -366.027 * row['S_chem_first']
        row['node'] = 8
    elif (row['Al_Bar_lf'] > 15.026) and (row['AL_chem_first'] <= 0.046) and (row['S_chem_first'] > 0.02) and (row['final_O2_ppm'] <= 980.5):
        row['AL_mat'] = -0.2894 * row['final_O2_ppm'] + 1.25 * row['LM_Start_Wt_lf'] + 0.0502 * row['1st_Probe_temp_lf'] + -0.0683 * row['Al_Bar_lf'] + -0.0091 * row['Lime_tap'] + 3294.5118 * row['AL_chem_first'] + 3035.8515 * row['S_chem_first']
        row['node'] = 9
    elif (row['Al_Bar_lf'] <= 15.026) and (row['AL_chem_first'] <= 0.032) and (row['S_chem_first'] <= 0.027) and (row['final_O2_ppm'] > 1144.5):
        row['AL_mat'] = -0.0417 * row['final_O2_ppm'] + -0.1288 * row['LM_Start_Wt_lf'] + 0.9894 * row['1st_Probe_temp_lf'] + -103.7983 * row['Al_Bar_lf'] + 0.0911 * row['Lime_tap'] + 1693.7519 * row['AL_chem_first'] + 4085.8512 * row['S_chem_first']
        row['node'] = 10
    elif (row['Al_Bar_lf'] <= 15.026) and (row['AL_chem_first'] > 0.032) and (row['Lime_tap'] <= 995.5) and (row['LM_Start_Wt_lf'] > 185.5):
        row['AL_mat'] = -0.0592 * row['final_O2_ppm'] + 0.5833 * row['LM_Start_Wt_lf'] + 1.1643 * row['1st_Probe_temp_lf'] + -119.9933 * row['Al_Bar_lf'] + 0.0059 * row['Lime_tap'] + 406.2502 * row['AL_chem_first'] + 453.8333 * row['S_chem_first']
        row['node'] = 11
    elif (row['Al_Bar_lf'] <= 15.026) and (row['AL_chem_first'] > 0.032) and (row['Lime_tap'] > 995.5) and (row['S_chem_first'] > 0.029):
        row['AL_mat'] = -0.95 * row['final_O2_ppm'] + 0.4265 * row['LM_Start_Wt_lf'] + -2.3793 * row['1st_Probe_temp_lf'] + 449.1525 * row['Al_Bar_lf'] + -0.2327 * row['Lime_tap'] + 4141.8124 * row['AL_chem_first'] + -51050.0 * row['S_chem_first']
        row['node'] = 12
    elif (row['Al_Bar_lf'] <= 15.026) and (row['AL_chem_first'] > 0.032) and (row['Lime_tap'] <= 995.5) and (row['LM_Start_Wt_lf'] <= 185.5):
        row['AL_mat'] = 0.1307 * row['final_O2_ppm'] + 5.0999 * row['LM_Start_Wt_lf'] + 0.5949 * row['1st_Probe_temp_lf'] + -133.0633 * row['Al_Bar_lf'] + -0.1308 * row['Lime_tap'] + 4123.9197 * row['AL_chem_first'] + 2242.8091 * row['S_chem_first']
        row['node'] = 13
    elif (row['Al_Bar_lf'] > 15.026) and (row['AL_chem_first'] > 0.046) and (row['AL_chem_first'] > 0.046) and (row['Lime_tap'] > 2927.5):
        row['AL_mat'] = -0.891 * row['final_O2_ppm'] + -2.3266 * row['LM_Start_Wt_lf'] + 1.5258 * row['1st_Probe_temp_lf'] + 13.1046 * row['Al_Bar_lf'] + -0.2811 * row['Lime_tap'] + 603.2872 * row['AL_chem_first'] + -5683.1937 * row['S_chem_first']
        row['node'] = 14
    elif (row['Al_Bar_lf'] > 15.026) and (row['AL_chem_first'] <= 0.046) and (row['S_chem_first'] <= 0.02) and (row['Lime_tap'] > 3088.5):
        row['AL_mat'] = 0.5715 * row['final_O2_ppm'] + 0.4728 * row['LM_Start_Wt_lf'] + 0.2037 * row['1st_Probe_temp_lf'] + 8.4861 * row['Al_Bar_lf'] + -0.1905 * row['Lime_tap'] + -10340.0 * row['AL_chem_first'] + -9121.9165 * row['S_chem_first']
        row['node'] = 15
    else:
        print("weak node")
        row['node'] = 0
        row['AL_mat'] = (10.0 * (row['LM_Start_Wt_lf'] + 1) * (al_lsl - row['AL_chem_first'] + 16 * fade_rate ))/0.6
        
    return [row['AL_mat'], row['node']]

In [258]:
for node in sorted(ads_test['Carbon_Node_2'].unique()):
# for node in [10,11]:
    print('node',node)
    model_test_1=ads_test[(ads_test['Carbon_Node_2']==node) ]
#     model_test_1=ads_test[(ads_test['Carbon_Node_2']==3) | (ads_test['Carbon_Node_2']==5)]
    #model_test_1=ads_test.copy()
    print(len(model_test_1))

    model_test_1.fillna(0, inplace = True)

    # ads preprocessing

    model_test_1['Al_Bar_lf'] = model_test_1['Al_Bar_lf'].clip(15,25)
    model_test_1['LM_Start_Wt_lf'] = model_test_1['LM_Start_Wt_lf'].clip(170,205)
    model_test_1['LTA_lf'] = model_test_1['LTA_lf'].clip(60)
    model_test_1['1st_Probe_temp_lf'] = model_test_1['1st_Probe_temp_lf'].clip(1550)
    model_test_1['final_O2_ppm'] = model_test_1['final_O2_ppm'].clip(700,1200)
    #model_test_1['Lime_tap'] = model_test_1['Lime_tap'].clip(500)


    ################ AL Prediction #################################

    # predicting aluminium material
#     model_test_1['al_predictions'] =  model_test_1.apply(cgcr_aluminium_prediction_new, axis = 1)
    model_test_1['al_predictions'] =  model_test_1.apply(cgcr_aluminium_prediction_original, axis = 1)

    # splitting the list of outputs into separate columns
    model_test_1['AL_KGS'] = model_test_1['al_predictions'].apply(lambda x: x[0])
    model_test_1['Carbon Node'] = model_test_1['al_predictions'].apply(lambda x: x[1])
    model_test_1['Oxygen Node'] = model_test_1['al_predictions'].apply(lambda x:x[2])
    #ads_test['AL_KGS_2'] = ads_test['al_predictions'].apply(lambda x:x[3])
    del model_test_1['al_predictions']

    #par=[0.855,0.0838,-1.3642,-0.036,-1413.6468,4535.899,-0.0013,0.1752,-17950.0,-1058.9498]
    #ads_test['AL_KGS'] =  par[0]* ads_test['LM_Start_Wt_lf'] +  par[1]* ads_test['1st_Probe_temp_lf'] + par[2] * ads_test['Al_Bar_lf'] +  par[3]* ads_test['Lime_tap'] +  par[4]* ads_test['AL_chem_first'] +  par[5]* ads_test['S_chem_first'] +  par[6]* ads_test['LTA_lf'] + par[7] * ads_test['SiMn_tap'] + par[8] * ads_test['SI_chem_first'] +  par[9]* ads_test['C_chem_first']
    # replacing -ve predictions as 0
    model_test_1['AL_KGS'][model_test_1['AL_KGS']<0]=0

    # predicting aluminium material in meters
    model_test_1['AL_MTS'] = model_test_1['AL_KGS'] * (3)


    model_test_1=model_test_1.merge(actual_al[['HEAT_NUMBER','Actual Al']],on='HEAT_NUMBER', how='left')
    model_test_1['Exclude Flag']=0
    model_test_1['Exclude Flag'][model_test_1['Actual Al'].isna()]=1
    model_test_1['Exclude Flag'][model_test_1['Actual Al']==0]=1
    model_test_1['Actual Al'][model_test_1['Actual Al']==0]=1
    model_test_1['Actual Al']=model_test_1['Actual Al'].fillna(1)
    model_test_1['Delta']=abs(model_test_1['AL_KGS']-model_test_1['Actual Al'])
    model_test_1['MAPE']=model_test_1['Delta']/model_test_1['Actual Al']
    model_test_1['Delta_flag']=0
    model_test_1['Delta_flag'][model_test_1['Delta']<=33]=1
    model_test_1['Delta_flag_2']=0
    model_test_1['Delta_flag_2'][model_test_1['Delta']<=50]=1
    ads_mape=model_test_1[model_test_1['Exclude Flag']==0]
    mape=ads_mape['MAPE'].mean()
    delta=ads_mape['Delta'].mean()
    print(len(ads_mape),'heats @',delta)
    print('Mape',mape)
    print('tolerance=',ads_mape['Delta_flag'].sum()/len(ads_mape))
    print('tolerance @ 50=',ads_mape['Delta_flag_2'].sum()/len(ads_mape))

    model_test_1['al_addition_equation'] =  model_test_1.apply(al_addition_equation, axis = 1)
    model_test_1['al_addition_kg'] = model_test_1['al_addition_equation'].apply(lambda x: x[0])
    model_test_1['al_addition_s_kg'] = model_test_1['al_addition_equation'].apply(lambda x: x[1])
    del model_test_1['al_addition_equation']



node 0
1488
1392 heats @ 75.09645201644416
Mape 0.5243115970110946
tolerance= 0.3074712643678161
tolerance @ 50= 0.444683908045977
node 1
499
419 heats @ 119.62560439392033
Mape 0.6395072290259116
tolerance= 0.30787589498806683
tolerance @ 50= 0.4152744630071599
node 2
461
419 heats @ 137.0849364390294
Mape 0.6130665449779514
tolerance= 0.2171837708830549
tolerance @ 50= 0.28878281622911695
node 3
628
557 heats @ 59.62902676718322
Mape 0.3860530400375889
tolerance= 0.4272890484739677
tolerance @ 50= 0.63016157989228
node 4
48
46 heats @ 90.07054899565217
Mape 0.5645840003311143
tolerance= 0.32608695652173914
tolerance @ 50= 0.45652173913043476
node 5
559
522 heats @ 71.23421569537842
Mape 0.24639333759969323
tolerance= 0.40229885057471265
tolerance @ 50= 0.5613026819923371
node 6
185
173 heats @ 93.37339225071497
Mape 0.4536226755880281
tolerance= 0.2774566473988439
tolerance @ 50= 0.4046242774566474
node 7
164
150 heats @ 84.31803856655367
Mape 0.25172096401369554
tolerance= 0.3133333

In [261]:
# for node in sorted(ads_test['Carbon_Node_2'].unique()):
#     model_test_1=ads_test[(ads_test['Carbon_Node_2']==3) | (ads_test['Carbon_Node_2']==5)]
model_test_1=ads_test.copy()
print(len(model_test_1))

model_test_1.fillna(0, inplace = True)

# ads preprocessing

model_test_1['Al_Bar_lf'] = model_test_1['Al_Bar_lf'].clip(15,25)
model_test_1['LM_Start_Wt_lf'] = model_test_1['LM_Start_Wt_lf'].clip(170,205)
model_test_1['LTA_lf'] = model_test_1['LTA_lf'].clip(60)
model_test_1['1st_Probe_temp_lf'] = model_test_1['1st_Probe_temp_lf'].clip(1550)
model_test_1['final_O2_ppm'] = model_test_1['final_O2_ppm'].clip(700,1200)
#model_test_1['Lime_tap'] = model_test_1['Lime_tap'].clip(500)


################ AL Prediction #################################

# predicting aluminium material
model_test_1['al_predictions'] =  model_test_1.apply(cgcr_aluminium_prediction_new, axis = 1)
# model_test_1['al_predictions'] =  model_test_1.apply(cgcr_aluminium_prediction_original, axis = 1)

# splitting the list of outputs into separate columns
model_test_1['AL_KGS'] = model_test_1['al_predictions'].apply(lambda x: x[0])
model_test_1['Carbon Node'] = model_test_1['al_predictions'].apply(lambda x: x[1])
model_test_1['Oxygen Node'] = model_test_1['al_predictions'].apply(lambda x:x[2])
#ads_test['AL_KGS_2'] = ads_test['al_predictions'].apply(lambda x:x[3])
del model_test_1['al_predictions']

#par=[0.855,0.0838,-1.3642,-0.036,-1413.6468,4535.899,-0.0013,0.1752,-17950.0,-1058.9498]
#ads_test['AL_KGS'] =  par[0]* ads_test['LM_Start_Wt_lf'] +  par[1]* ads_test['1st_Probe_temp_lf'] + par[2] * ads_test['Al_Bar_lf'] +  par[3]* ads_test['Lime_tap'] +  par[4]* ads_test['AL_chem_first'] +  par[5]* ads_test['S_chem_first'] +  par[6]* ads_test['LTA_lf'] + par[7] * ads_test['SiMn_tap'] + par[8] * ads_test['SI_chem_first'] +  par[9]* ads_test['C_chem_first']
# replacing -ve predictions as 0
model_test_1['AL_KGS'][model_test_1['AL_KGS']<0]=0

# predicting aluminium material in meters
model_test_1['AL_MTS'] = model_test_1['AL_KGS'] * (3)


model_test_1=model_test_1.merge(actual_al[['HEAT_NUMBER','Actual Al']],on='HEAT_NUMBER', how='left')
model_test_1['Exclude Flag']=0
model_test_1['Exclude Flag'][model_test_1['Actual Al'].isna()]=1
model_test_1['Exclude Flag'][model_test_1['Actual Al']==0]=1
model_test_1['Actual Al'][model_test_1['Actual Al']==0]=1
model_test_1['Actual Al']=model_test_1['Actual Al'].fillna(1)
model_test_1['Delta']=abs(model_test_1['AL_KGS']-model_test_1['Actual Al'])
model_test_1['MAPE']=model_test_1['Delta']/model_test_1['Actual Al']
model_test_1['Delta_flag']=0
model_test_1['Delta_flag'][model_test_1['Delta']<=33]=1
model_test_1['Delta_flag_2']=0
model_test_1['Delta_flag_2'][model_test_1['Delta']<=50]=1
ads_mape=model_test_1[model_test_1['Exclude Flag']==0]
mape=ads_mape['MAPE'].mean()
delta=ads_mape['Delta'].mean()
print(len(ads_mape),'heats @',delta)
print('Mape',mape)
print('tolerance=',ads_mape['Delta_flag'].sum()/len(ads_mape))
print('tolerance @ 50=',ads_mape['Delta_flag_2'].sum()/len(ads_mape))

model_test_1['al_addition_equation'] =  model_test_1.apply(al_addition_equation, axis = 1)
model_test_1['al_addition_kg'] = model_test_1['al_addition_equation'].apply(lambda x: x[0])
model_test_1['al_addition_s_kg'] = model_test_1['al_addition_equation'].apply(lambda x: x[1])
del model_test_1['al_addition_equation']



4969
4540 heats @ 71.41745732515636
Mape 0.39651076621641457
tolerance= 0.36211453744493394
tolerance @ 50= 0.5118942731277533


In [262]:
model_test_1.to_excel(r'C:\Users\khera siddharth\Desktop\Desktop_Folder\Case Work\JSW_dolvi\DeS\Working files\Model Prediction\Feb 23\output\testing\iterations\all_heats_new eqn.xlsx')

In [130]:
from sklearn.linear_model import LinearRegression

ads_regression=ads_mape.copy()
# create linear regression object
lr = LinearRegression()
#lr.fit(ads_regression[['AL_KGS']], ads_regression['Actual Al'])
lr.fit(ads_regression[['Actual Al']], ads_regression['AL_KGS'])

print(lr.intercept_)
print(lr.coef_[0])

ads_regression['intercept']=lr.intercept_
ads_regression['coeff']=lr.coef_[0]
#ads_regression['al_cal']=ads_regression['AL_KGS']*ads_regression['coeff']+ads_regression['intercept']

ads_regression['al_cal']=ads_regression['AL_KGS']*0.75 + 20 #ads_regression['intercept']
ads_regression['Delta']=abs(ads_regression['al_cal']-ads_test['Actual Al'])
ads_regression['MAPE']=ads_regression['Delta']/ads_regression['Actual Al']
ads_regression['Delta_flag']=0
ads_regression['Delta_flag'][ads_regression['Delta']<=40]=1
mape=ads_regression['MAPE'].mean()
print(len(ads_regression),'heats @',mape)
print('tolerance=',ads_regression['Delta_flag'].sum()/len(ads_regression))


117.77499565561672
0.11288824327494525
419 heats @ 0.5114166765418452
tolerance= 0.45584725536992843


In [95]:
ads_mape[[,'Actual Al']]

0.3818615751789976

In [98]:
ads_test[ads_test['HEAT_NUMBER']==22100338]

,HEAT_NUMBER,AGGREGATE,AL_chem_first,S_chem_first,C_chem_first,SI_chem_first,Month,STATION,MSG_TIME_STAMP,GRADE,...,Carbon Node,Oxygen Node,AL_MTS,Actual Al,Exclude Flag,Delta,MAPE,Delta_flag,al_addition_kg,al_addition_s_kg


In [24]:
ads_test

,HEAT_NUMBER,AGGREGATE,AL_chem_first,S_chem_first,C_chem_first,SI_chem_first,Month,STATION,MSG_TIME_STAMP,GRADE,...,Carbon_Node,Oxygen_Node,AL_KGS,Carbon Node,Oxygen Node,AL_MTS,Actual Al,Delta,MAPE,Delta_flag
0,22400754,LF4,0.053,0.017,0.032,0.003,2,4.0,2022-02-10 20:16:53.637,JDHCG04ACN,...,4.0,0.0,154.218609,4,15,462.655827,NaN,NaN,NaN,0
1,22200708,LF1,0.044,0.013,0.034,0.006,2,1.0,2022-02-10 10:42:53.057,JDHCG04ACN,...,4.0,0.0,99.436324,2,2,298.308973,NaN,NaN,NaN,0
2,22200707,LF2,0.048,0.020,0.029,0.001,2,5.0,2022-02-10 09:55:53.033,JDHCG04ACN,...,4.0,0.0,157.562421,8,2,472.687263,NaN,NaN,NaN,0
3,22200706,LF5,0.016,0.019,0.030,0.005,2,5.0,2022-02-10 09:06:52.967,JDHCG04ACN,...,NaN,NaN,239.381344,7,4,718.144031,NaN,NaN,NaN,0
4,22200705,LF1,0.037,0.005,0.038,0.010,2,1.0,2022-02-10 07:29:52.893,JDHCG04ACN,...,7.0,4.0,41.617727,2,2,124.853180,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4642,21105539,LF1,0.051,0.012,0.045,0.161,12,1.0,2021-12-10 14:39:00.423,JDHST46ILN,...,12.0,0.0,104.692283,1,0,314.076850,231.0,126.307717,0.546787,0
4643,21305604,LF1,0.040,0.023,0.040,0.061,12,1.0,2021-12-05 10:58:18.410,JDHST46ILN,...,7.0,0.0,217.650562,7,0,652.951686,198.0,19.650562,0.099245,1
4644,21105446,LF1,0.025,0.026,0.057,0.114,12,1.0,2021-12-05 04:14:18.047,JDHST46ILN,...,12.0,0.0,123.529946,12,0,370.589839,206.9,83.370054,0.402949,0
4645,21105435,LF1,0.040,0.030,0.045,0.192,12,1.0,2021-12-04 11:56:49.277,JDHPP65MSZ,...,7.0,0.0,192.835673,7,0,578.507018,297.0,104.164327,0.350722,0


In [87]:
ads_test.to_excel(r'C:\Users\khera siddharth\Desktop\Desktop_Folder\Case Work\JSW_dolvi\DeS\Working files\Model Prediction\Feb 21\output\node 4\node4_c and o nodes.xlsx')